# Identify DOSTA Sensors with Missing Two-Point Calibrations

During a review of the dissolved oxygen data, it was discovered that there was an error in how the instrument calibration coefficients were being applied. The two-point calibration values, supplied by the vendor if a multipoint calibration was not warranted, were not being passed to the equation used to convert the raw measurements to dissolved oxygen. The resulting calculated dissolved oxygen values were incorrect. The error in the formulation of the equation was identified and corrected, with the correction going into effect on September 10, 2020.

The code below identifies the instruments impacted using the OOI M2M API to access calibration values for the sensors from the Asset Management database and determine which sensors had two-point calibration values. The default values are set to a slope of 1 and offset of 0. If the vendor applied a two-point calibration, those values will be different. The error in the code always assumed the default values for the two-point calibration.

In [1]:
import pandas as pd

from datetime import datetime, timedelta
from ooi_data_explorations.common import list_deployments, get_deployment_dates, \
    get_calibrations_by_refdes, get_vocabulary

In [2]:
def missing_two_point(sites, node, sensor):
    """
    Use the OOI M2M API to locate the DOSTA sensors that had nondefault two-point
    calibration values; default is a slope of 1 and offset of 0. These are the
    sensors that would have had miscalculated dissolved oxygen data as the 
    function was always assuming the default values.
    
    :param site: Site name to query
    :param node: Node name to query
    :param sensor: Sensor name to query
    :return: pandas dataframe listing affected sensors
    """    
    # create an empty pandas dataframe
    data = pd.DataFrame(columns=['Array', 'Platform', 'Node', 'Instrument', 'RefDes', 'Asset_ID', 
                                 'Serial Number', 'deployment', 'gitHub_changeDate', 'file', 'URL',
                                 'changeType', 'dateRangeStart', 'dateRangeEnd', 'annotation'])
    
    # loop through the sites
    for site in sites:
        # for each site, loop through the deployments
        deployments = list_deployments(site, node, sensor)
        for deploy in deployments:
            # get the deployment dates and convert to a datetime object
            start, stop = get_deployment_dates(site, node, sensor, deploy)
            vocab = get_vocabulary(site, node, sensor)
            start = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S.000Z')
            stop = datetime.strptime(stop, '%Y-%m-%dT%H:%M:%S.000Z')

            # advance the start time by 30 days to ensure we only get calibration
            # data for the deployment of interest (exclude potentially overlapping)
            adj_start = start + timedelta(days=30)
            adj_stop = start + timedelta(days=31)

            # use the site, node, sensor and advanced start and stop dates to
            # access the sensor calibration data (using 1 day in the middle
            # of the deployment limits the response to just this deployment)
            cal = get_calibrations_by_refdes(site, node, sensor,
                                             adj_start.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
                                             adj_stop.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

            # extract the two-point calibration values
            two_point = cal[0]['sensor']['calibration'][1]['calData'][0]['value']

            # check to see if a two-point calibration was available
            if two_point != [0.0, 1.0]:
                # if so, add the information to the dataframe
                percent_error = abs(100 - (two_point[0] + two_point[1] * 100))
                annotation = (('ALGORITHM CORRECTION: During a review of the dissolved oxygen data, it was ' +
                               'discovered that there was an error in how the instrument calibration ' +
                               'coefficients were being applied. The two-point calibration values, supplied by ' +
                               'the vendor if a multipoint calibration was not warranted, were not being passed ' +
                               'to the equation used to convert the raw measurements to dissolved oxygen. The ' +
                               'resulting calculated dissolved oxygen values were incorrect. The error in the ' +
                               'formulation of the equation was identified and corrected, with the correction ' +
                               'going into effect on 2020-09-10. Users who have requested data for this sensor ' +
                               '({0}) prior to 2020-09-10 for deployment {1} between {2} and {3} are encouraged ' +
                               'to re-download the data. The estimated error in the dissolved oxygen ' +
                               'calculation in this instance is {4:.2f} percent.').format(cal[0]['referenceDesignator'], 
                                                                                          deploy, start, stop, 
                                                                                          percent_error))
                data = data.append({'Array': vocab[0]['tocL1'],  
                                    'Platform': vocab[0]['tocL2'] , 
                                    'Node': vocab[0]['tocL3'], 
                                    'Instrument': vocab[0]['instrument'], 
                                    'RefDes': cal[0]['referenceDesignator'], 
                                    'Asset_ID': cal[0]['sensor']['uid'],
                                    'Serial Number': cal[0]['sensor']['serialNumber'], 
                                    'deployment': deploy, 
                                    'gitHub_changeDate': '2020-09-10', 
                                    'file': 'ParameterFunctions.csv', 
                                    'URL': 'https://github.com/oceanobservatories/preload-database/commit/c07c9229d01040da16e2cf6270c7180d4ed57f20',
                                    'changeType': 'algorithmCorrection', 
                                    'dateRangeStart': start, 
                                    'dateRangeEnd': stop, 
                                    'annotation': annotation}, ignore_index=True)
                
    # return the results
    return data

## Coastal Endurance Array

Two sets of DOSTA sensors were potentially impacted by the error in the Coastal Endurance (CE) Array: the sensors mounted on the near-surface instrument frame (NSIF) for the shelf and offshore coastal surface moorings, and the sensors on the coastal surface piercing profiler (CSPP). 

In [3]:
# find the endurance instruments affected
sites = ['CE02SHSM', 'CE04OSSM', 'CE07SHSM', 'CE09OSSM']
node = 'RID27'
sensor = '04-DOSTAD000'
nsif = missing_two_point(sites, node, sensor)
sites = ['CE01ISSP', 'CE02SHSP', 'CE06ISSP', 'CE07SHSP']
node = 'SP001'
sensor = '01-DOSTAJ000'
cspp = missing_two_point(sites, node, sensor)
endurance = pd.concat([nsif, cspp], ignore_index=True)
endurance

,Array,Platform,Node,Instrument,RefDes,Asset_ID,Serial Number,deployment,gitHub_changeDate,file,URL,changeType,dateRangeStart,dateRangeEnd,annotation
0,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00313,313,3,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-05-17 21:23:00,2016-09-28 15:52:00,ALGORITHM CORRECTION: During a review of the d...
1,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00220,220,5,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-04-20 17:23:00,2017-10-15 19:06:32,ALGORITHM CORRECTION: During a review of the d...
2,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00484,484,6,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-10-10 02:21:08,2018-04-04 13:57:00,ALGORITHM CORRECTION: During a review of the d...
3,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00223,223,7,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-04-03 01:17:00,2018-09-25 16:16:00,ALGORITHM CORRECTION: During a review of the d...
4,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00134,134,9,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2019-04-19 23:50:00,2019-10-22 15:32:00,ALGORITHM CORRECTION: During a review of the d...
5,Coastal Endurance,Oregon Shelf Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE02SHSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00292,292,11,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2020-07-10 02:18:00,2020-10-10 14:55:14,ALGORITHM CORRECTION: During a review of the d...
6,Coastal Endurance,Oregon Offshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE04OSSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00220,220,2,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-05-16 04:48:00,2016-10-01 15:21:00,ALGORITHM CORRECTION: During a review of the d...
7,Coastal Endurance,Oregon Offshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE04OSSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00482,482,3,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-10-01 00:44:00,2017-04-21 15:35:00,ALGORITHM CORRECTION: During a review of the d...
8,Coastal Endurance,Oregon Offshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE04OSSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00135,135,4,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-04-21 01:44:00,2017-10-13 16:28:34,ALGORITHM CORRECTION: During a review of the d...
9,Coastal Endurance,Oregon Offshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CE04OSSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00485,485,5,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-10-11 23:08:23,2018-04-03 20:27:00,ALGORITHM CORRECTION: During a review of the d...


In [4]:
endurance.to_csv('endurance.dosta.changelog.csv')

## Coastal Pioneer Array

Three sets of DOSTA sensors were potentially impacted by the error in the Coastal Pioneer (CP) Array: the sensors mounted on the near-surface instrument frame (NSIF) and the multi-function node (MFN) for the coastal surface moorings, and the sensors on the coastal surface piercing profiler (CSPP). 

In [5]:
# find the pioneer instruments affected
sites = ['CP01CNSM', 'CP03ISSM', 'CP04OSSM']
sensor = '04-DOSTAD000'
nsif = missing_two_point(sites, 'RID27', sensor)
mfn = missing_two_point(sites, 'MFD37', sensor)
sites = ['CP01CNSP', 'CP03ISSP']
node = 'SP001'
sensor = '01-DOSTAJ000'
cspp = missing_two_point(sites, node, sensor)
pioneer = pd.concat([nsif, mfn, cspp], ignore_index=True)
pioneer

,Array,Platform,Node,Instrument,RefDes,Asset_ID,Serial Number,deployment,gitHub_changeDate,file,URL,changeType,dateRangeStart,dateRangeEnd,annotation
0,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00503,503,6,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-10-13 18:36:00,2017-06-09 16:05:00,ALGORITHM CORRECTION: During a review of the d...
1,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00515,515,7,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-06-09 14:24:00,2017-11-01 20:33:00,ALGORITHM CORRECTION: During a review of the d...
2,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00377,377,8,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-10-29 14:15:00,2018-03-29 19:37:00,ALGORITHM CORRECTION: During a review of the d...
3,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00128,128,9,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-03-24 21:32:00,2018-10-29 12:31:00,ALGORITHM CORRECTION: During a review of the d...
4,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00392,392,10,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-10-30 01:48:00,2019-04-07 18:08:00,ALGORITHM CORRECTION: During a review of the d...
5,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00456,456,11,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2019-04-06 14:35:00,2019-09-26 17:15:00,ALGORITHM CORRECTION: During a review of the d...
6,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP01CNSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00497,497,12,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2019-09-27 18:30:00,2020-10-10 14:56:19,ALGORITHM CORRECTION: During a review of the d...
7,Coastal Pioneer,Inshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP03ISSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00514,514,5,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-10-11 13:39:00,2017-06-15 15:56:00,ALGORITHM CORRECTION: During a review of the d...
8,Coastal Pioneer,Inshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP03ISSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00378,378,6,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-06-15 13:43:00,2017-11-01 14:17:00,ALGORITHM CORRECTION: During a review of the d...
9,Coastal Pioneer,Inshore Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,CP03ISSM-RID27-04-DOSTAD000,CGINS-DOSTAD-00461,461,7,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-11-01 12:22:00,2018-03-28 16:56:00,ALGORITHM CORRECTION: During a review of the d...


In [6]:
pioneer.to_csv('pioneer.dosta.changelog.csv')

## Global Arrays (Argentine Basin, Irminger Sea, Southern Ocean and Station Papa)

4 sets of DOSTA sensors were potentially impacted by the error in the Global Arrays: the sensors mounted on the subsurface plate of the buoy and the near-surface instrument frame (NSIF), the sensors connected to the CTDBP on the mooring riser, and the sensors on the flanking mooring subsurface sphere.

In [7]:
sites = ['GA01SUMO', 'GI01SUMO', 'GS01SUMO']
buoy = missing_two_point(sites, 'SBD11', '04-DOSTAD000')
nsif = missing_two_point(sites, 'RID16', '06-DOSTAD000')
imm = [missing_two_point(sites, 'RII11', '02-DOSTAD031'),
       missing_two_point(sites, 'RII11', '02-DOSTAD032'),
       missing_two_point(sites, 'RII11', '02-DOSTAD033')]
sites = ['GA03FLMA', 'GA03FLMB', 'GI03FLMA', 'GI03FLMB', 'GP03FLMA', 'GP03FLMB', 'GS03FLMA', 'GS03FLMB']
sphere = missing_two_point(sites, 'RIS01', '03-DOSTAD000')
garray = pd.concat([buoy, nsif, pd.concat(imm), sphere], ignore_index=True)
garray

,Array,Platform,Node,Instrument,RefDes,Asset_ID,Serial Number,deployment,gitHub_changeDate,file,URL,changeType,dateRangeStart,dateRangeEnd,annotation
0,Global Argentine Basin,Apex Surface Mooring,Surface Buoy,Dissolved Oxygen,GA01SUMO-SBD11-04-DOSTAD000,CGINS-DOSTAD-00379,379,3,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-10-27 01:47:00,2018-01-14 10:24:00,ALGORITHM CORRECTION: During a review of the d...
1,Global Irminger Sea,Apex Surface Mooring,Surface Buoy,Dissolved Oxygen,GI01SUMO-SBD11-04-DOSTAD000,CGINS-DOSTAD-00495,495,4,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-08-05 18:17:00,2017-10-12 09:07:00,ALGORITHM CORRECTION: During a review of the d...
2,Global Irminger Sea,Apex Surface Mooring,Surface Buoy,Dissolved Oxygen,GI01SUMO-SBD11-04-DOSTAD000,CGINS-DOSTAD-00211,211,5,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-06-08 17:21:00,2019-08-09 08:04:00,ALGORITHM CORRECTION: During a review of the d...
3,Global Irminger Sea,Apex Surface Mooring,Surface Buoy,Dissolved Oxygen,GI01SUMO-SBD11-04-DOSTAD000,CGINS-DOSTAD-00127,127,6,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2019-08-05 15:29:00,2020-08-26 10:41:00,ALGORITHM CORRECTION: During a review of the d...
4,Global Southern Ocean,Apex Surface Mooring,Surface Buoy,Dissolved Oxygen,GS01SUMO-SBD11-04-DOSTAD000,CGINS-DOSTAD-00465,465,4,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-12-04 17:10:40,2020-01-20 10:01:00,ALGORITHM CORRECTION: During a review of the d...
5,Global Argentine Basin,Apex Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,GA01SUMO-RID16-06-DOSTAD000,CGINS-DOSTAD-00382,382,3,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-10-27 01:47:00,2018-01-14 10:24:00,ALGORITHM CORRECTION: During a review of the d...
6,Global Irminger Sea,Apex Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,GI01SUMO-RID16-06-DOSTAD000,CGINS-DOSTAD-00497,497,4,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2017-08-05 18:17:00,2017-10-12 09:07:00,ALGORITHM CORRECTION: During a review of the d...
7,Global Irminger Sea,Apex Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,GI01SUMO-RID16-06-DOSTAD000,CGINS-DOSTAD-00212,212,5,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2018-06-08 17:21:00,2019-08-09 08:04:00,ALGORITHM CORRECTION: During a review of the d...
8,Global Irminger Sea,Apex Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,GI01SUMO-RID16-06-DOSTAD000,CGINS-DOSTAD-00129,129,6,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2019-08-05 15:29:00,2020-08-26 10:41:00,ALGORITHM CORRECTION: During a review of the d...
9,Global Southern Ocean,Apex Surface Mooring,Near Surface Instrument Frame,Dissolved Oxygen,GS01SUMO-RID16-06-DOSTAD000,CGINS-DOSTAD-00428,428,3,2020-09-10,ParameterFunctions.csv,https://github.com/oceanobservatories/preload-...,algorithmCorrection,2016-11-25 01:11:00,2018-12-09 16:42:00,ALGORITHM CORRECTION: During a review of the d...


In [8]:
garray.to_csv('globals.dosta.changelog.csv')